In [48]:
import numpy as np
import pandas as pd

In [26]:
#Hente tickere og grunnleggende informasjon om alle teknologiselskaper
df = pd.read_html("https://finance.yahoo.com/screener/predefined/ms_technology?count=100&offset=0")[0]
for i in range(1,5):
    df = df.append(pd.read_html("https://finance.yahoo.com/screener/predefined/ms_technology?count=100&offset={number}00".format(number = str(i)))[0])

In [29]:
df.index = df.Symbol

In [30]:
df

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
Symbol,,,,,,,,,,
AAPL,AAPL,Apple Inc.,125.12,-2.67,-2.09%,97.447M,106.409M,2.101T,33.94,NaN
MSFT,MSFT,Microsoft Corporation,233.87,-3.07,-1.30%,21.811M,28.804M,1.764T,34.87,NaN
TSM,TSM,Taiwan Semiconductor Manufacturing Company Lim...,126.00,-3.13,-2.42%,7.04M,11.616M,552.04B,56.50,NaN
NVDA,NVDA,NVIDIA Corporation,536.25,-17.42,-3.15%,6.493M,7.504M,331.939B,77.72,NaN
INTC,INTC,Intel Corporation,61.24,-1.64,-2.61%,22.836M,38.817M,248.818B,12.40,NaN
...,...,...,...,...,...,...,...,...,...,...
IDBA,IDBA,IDEX Biometrics ASA,27.76,-0.44,-1.56%,13807,19020,328.97M,NaN,NaN
IIVIP,IIVIP,II-VI Incorporated,352.46,-5.60,-1.56%,553,12355,NaN,NaN,NaN
SABRP,SABRP,Sabre Corporation,172.39,-9.68,-5.32%,25343,12508,NaN,NaN,NaN


In [34]:
#Hente informasjon fra statistics-fanen (bla. short %) for alle tickerene
tickers = df.Symbol
dic = {}
for ticker in tickers:
    dfs = pd.read_html("https://finance.yahoo.com/quote/{sym}/key-statistics?p={sym}".format(sym = ticker))
    dic[ticker] = dfs[2].iloc[:,1]
    variables = dfs[2].iloc[:,0]

In [36]:
#organisere informasjonen i dataframe
df2 = pd.DataFrame(index = variables.values)
for ticker in tickers:
    df2[ticker] = dic[ticker].values

In [42]:
#slå sammen med dataframe med grunnleggende informasjon
super_df = df.join(df2.T).copy()

In [ ]:
#lagre som excel
super_df.to_excel('tech_short_dataset.xlsx')

In [77]:
#Velge ut viktigste variable og droppe missing
super_df2 = super_df[['Symbol', 'Name', 'Price (Intraday)', 'Change', '% Change', 'Volume',
       'Avg Vol (3 month)', 'Market Cap', 'Avg Vol (10 day) 3', 'Shares Outstanding 5',
       'Float', '% Held by Insiders 1', '% Held by Institutions 1',
       'Shares Short (Feb 11, 2021) 4', 'Short Ratio (Feb 11, 2021) 4',
       'Short % of Float (Feb 11, 2021) 4',
       'Short % of Shares Outstanding (Feb 11, 2021) 4',
       'Shares Short (prior month Jan 14, 2021) 4']].dropna().copy()

In [78]:
#gjøre om k->1000, M->1000000 osv
cols_fix = ['Volume',
       'Avg Vol (3 month)', 'Market Cap', 'Avg Vol (10 day) 3', 'Shares Outstanding 5',
       'Float', 'Shares Short (Feb 11, 2021) 4',
       'Shares Short (prior month Jan 14, 2021) 4']
for col in cols_fix:
    super_df2[col] = super_df2[col].replace({'k': '*1e3', 'M': '*1e6', 'B': '*1e9', 'T': '*1e12'}, regex=True).map(pd.eval).astype(np.int64)

In [80]:
super_df2.to_csv('tech_short_dataset_clean.csv')